<a href="https://colab.research.google.com/github/pandasmc/my-bigcontest-app/blob/data/%EC%A0%9C%EB%AF%B8%EB%82%98%EC%9D%B4%26%EC%8A%A4%ED%8A%B8%EB%A6%BC%EB%A6%BF_251021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# 1-1. 구글 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# 1-2. 라이브러리 불러오기
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import folium # 지도 시각화를 위한 라이브러리
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False # 마이너스 부호 깨짐 방지

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
!pip install streamlit
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 96.2 MB/s eta 0:00:00


In [9]:
df = pd.read_csv('/content/drive/MyDrive/01. 공모전/03. 빅콘테스트/data/최종데이터.csv', encoding='cp949')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4185 entries, 0 to 4184
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  4185 non-null   int64 
 1   가맹점ID       4185 non-null   object
 2   가맹점명        4185 non-null   object
 3   맞춤형설명       4185 non-null   object
 4   업종          4185 non-null   object
 5   개설일         4185 non-null   object
 6   폐업일         127 non-null    object
 7   주소          4185 non-null   object
 8   상권          4185 non-null   object
dtypes: int64(1), object(8)
memory usage: 294.4+ KB


In [12]:
df.drop(columns='Unnamed: 0', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4185 entries, 0 to 4184
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   가맹점ID   4185 non-null   object
 1   가맹점명    4185 non-null   object
 2   맞춤형설명   4185 non-null   object
 3   업종      4185 non-null   object
 4   개설일     4185 non-null   object
 5   폐업일     127 non-null    object
 6   주소      4185 non-null   object
 7   상권      4185 non-null   object
dtypes: object(8)
memory usage: 261.7+ KB


# GEMINI 2.5 Pro 연동하기

In [14]:
def generate_prompt(store_name, description, industry, open_date, close_date):
    prompt = f"""
당신은 소상공인을 위한 AI 전략 컨설턴트입니다.

가맹점명: {store_name}
업종: {industry}
개설일: {open_date}
폐업일: {close_date if close_date else '폐업하지 않음'}
맞춤형설명: {description}

위 정보를 바탕으로, 해당 가맹점의 운영 전략, 고객 타겟팅, 매출 개선 방향, 폐업 위험 요인 등을 체계적으로 분석해 주세요.
"""
    return prompt.strip()

In [15]:
data = df.copy()
code = """
import streamlit as st
import pandas as pd

# 데이터 불러오기
data = pd.read_csv("final.csv")  # 실제 파일명으로 바꿔주세요

def generate_prompt(store_name, description, industry, open_date, close_date):
    prompt = f\"\"\"
당신은 소상공인을 위한 AI 전략 컨설턴트입니다.

가맹점명: {store_name}
업종: {industry}
개설일: {open_date}
폐업일: {close_date if close_date else '폐업하지 않음'}
맞춤형설명: {description}

위 정보를 바탕으로, 해당 가맹점의 운영 전략, 고객 타겟팅, 매출 개선 방향, 폐업 위험 요인 등을 체계적으로 분석해 주세요.
\"\"\"
    return prompt.strip()

st.title("🧠 AI 비서: 맞춤형 전략 리포트")

search = st.text_input("🔍 가맹점명을 입력하세요")
filtered = data[data['가맹점명'].str.contains(search, case=False, na=False)]

if not filtered.empty:
    selected = st.selectbox("가맹점을 선택하세요", filtered['가맹점명'].unique())
    store_info = data[data['가맹점명'] == selected].iloc[0]

    # 프롬프트 생성
    prompt = generate_prompt(
        store_name=store_info['가맹점명'],
        description=store_info['맞춤형설명'],
        industry=store_info['업종'],
        open_date=store_info['개설일'],
        close_date=store_info['폐업일']
    )

    st.markdown("## 🧾 AI 전략 리포트")
    st.text_area("Gemini 프롬프트", prompt, height=300)

    # 여기에 Gemini API 연동해서 응답 출력 가능
    # response = gemini.generate(prompt)
    # st.markdown(response)
else:
    st.warning("해당 가맹점을 찾을 수 없습니다.")
"""

with open("/content/drive/MyDrive/01. 공모전/03. 빅콘테스트/data/app.py", "w", encoding="utf-8") as f:
    f.write(code)

2025-10-21 02:27:09.584 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-21 02:27:09.585 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-21 02:27:09.587 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-21 02:27:09.588 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-21 02:27:09.590 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-21 02:27:09.591 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-21 02:27:09.593 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-21 02:27:09.595 Session state does not function when running a script without `streamlit run`
2025-10-21 02:27

In [6]:
code = """
import streamlit as st

st.title("🧠 AI 비서: 가맹점 전략 리포트")
st.write("가맹점명을 입력하면 맞춤형 전략을 제공합니다.")
"""

with open("app.py", "w", encoding="utf-8") as f:
    f.write(code)

In [1]:
# 한글 깨짐 방지
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (6,933 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 126675 files and dire